In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784")

In [2]:
X_all = mnist.data.values
y_all = mnist.target
import pandas as pd

In [3]:
import numpy as np
X0 = X_all[np.array(np.where(y_all == '0'))]
X1 = X_all[np.array(np.where(y_all == '1'))]
X0 = X0[0,:,:]
X1 = X1[0,:,:]
y0 = np.zeros(X0.shape[0])
y1 = np.ones(X1.shape[0])

In [4]:
X = np.concatenate((X0, X1), axis = 0)
y = np.concatenate((y0, y1))

In [5]:
print(np.shape(X))
print(np.shape(y))

(14780, 784)
(14780,)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 0. 0. ... 1. 1. 1.]


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=2000)

In [8]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score

logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)

LogisticRegression(C=100000.0)

In [17]:
print(logreg.coef_)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.24398578e-06
  -4.43537814e-06 -1.82323844e-06  1.78817616e-06 -9.16874916e-04
  -2.51776545e-03 -1.19774896e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.08403028e-07
  -1.75311

In [18]:
print(logreg.intercept_)

[0.00034602]


In [48]:
def sigmoid(S):
    return 1/(1 + np.exp(-S))

def prob(w, X):
    return sigmoid(X.dot(w))

def loss(w, X, y, lam):
    z = prob(w, X)
    return -np.mean(y*np.log(z) + (1-y)*np.log(1-z)) + 0.5*lam/X.shape[0]*np.sum(w*w)

import warnings

#suppress warnings
warnings.filterwarnings('ignore')


In [49]:
def logistic_regression(w_init, X, y, lam = 0.001, lr = 0.1, nepoches = 2000):
    # lam - reg paramether, lr - learning rate, nepoches - number of epoches
    N, d = X.shape[0], X.shape[1]
    w = w_old = w_init
    loss_hist = [loss(w_init, X, y, lam)]
    ep = 0
    while ep < nepoches:
        ep += 1
        mix_ids = np.random.permutation(N)
        for i in mix_ids:
            xi = X[i]
            yi = y[i]
            zi = sigmoid(xi.dot(w))
            w = w - lr*((zi - yi)*xi + lam*w)
        loss_hist.append(loss(w, X, y, lam))
        if np.linalg.norm(w - w_old)/d < 1e-6:
            break
        w_old = w
    return w, loss_hist

# bias trick
N = 12780
Xbar = np.concatenate((X_train, np.ones((N, 1))), axis = 1)
w_init = np.random.randn(Xbar.shape[1])
lam = 0.0001
w, loss_hist = logistic_regression(w_init, Xbar, y_train, lam, lr = 0.05, nepoches = 500)
print(w)
print(loss(w, Xbar, y_train, lam))

[ 1.40340821e-14  5.68733469e-15  1.80496306e-14  7.76941964e-15
  5.56981764e-15 -6.65084314e-16  1.16158524e-14  9.17840744e-15
  2.60944840e-15  5.03599243e-15 -3.52766483e-15  2.68090542e-15
 -3.25391262e-15  7.46979018e-15 -3.23257279e-15  1.63576246e-15
 -1.60667614e-14  1.51466075e-14  1.72160373e-14  5.83634902e-15
  1.22804761e-14 -1.70133182e-14 -7.85927456e-15 -2.50086963e-14
  3.84275132e-15 -9.08466024e-16  1.29342911e-14 -1.55254710e-14
  5.87179702e-15 -1.00797005e-14 -7.95686561e-15  7.08539491e-16
 -9.13564611e-15 -3.40286495e-15  1.34633448e-14  8.11400650e-15
 -1.67179986e-14  1.28600408e-14  3.29910336e-14  4.19128709e-15
  1.06928226e-14 -1.24374239e-14 -3.24305798e-14 -1.25600770e-06
 -3.43308773e-06 -1.63281002e-06 -1.20721386e-15  9.37170201e-16
  5.97348932e-15  4.98524608e-15 -1.62871461e-14 -1.71457860e-15
 -9.61409667e-15 -5.91699863e-15  3.19050081e-14 -1.26203982e-15
 -1.62552933e-14  1.20653864e-14 -2.63950696e-14  2.06468280e-15
 -3.43507539e-15  3.82480